In [13]:
import os
import sys
import gensim
import pandas as pd
import re
import pickle
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','<number>',text)
    return text

In [3]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments['content'] = clean_train_comments['content'].apply(preprocess)
clean_train_comments.head()

,content,polarity
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral
2,kasus toko obat digerebek fpi propam akan peri...,neutral
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral
4,minyak jarak castor oil <number> ml,neutral


In [4]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments.dtypes
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

,content,polarity,tokens,sentiment
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral,"[gerindra, alihkan, rekomendasi, ke, agus, an,...",1
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral,"[cuci, tangan, pakai, sunlight, stelah, itu, p...",1
2,kasus toko obat digerebek fpi propam akan peri...,neutral,"[kasus, toko, obat, digerebek, fpi, propam, ak...",1
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral,"[menkeu, melemah, nya, rupiah, lebih, berpenga...",1
4,minyak jarak castor oil <number> ml,neutral,"[minyak, jarak, castor, oil, number, ml]",1


In [5]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments['content'] = clean_test_comments['content'].apply(preprocess)
clean_test_comments.head()

,content,polarity
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [6]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments.dtypes
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

,content,polarity,tokens,sentiment
0,kemarin gue datang ke tempat makan baru yang a...,negative,"[kemarin, gue, datang, ke, tempat, makan, baru...",0
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,"[kayak, nya, sih, gue, tidak, akan, mau, balik...",0
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ...",0
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,"[ini, pertama, kalinya, gua, ke, bank, buat, n...",0
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,"[waktu, sampai, dengan, gue, pernah, disuruh, ...",0


In [7]:
x_train = clean_train_comments['content']
x_validation = clean_test_comments['content']
y_train = clean_train_comments['sentiment']
y_validation = clean_test_comments['sentiment']

In [8]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [9]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 700

In [14]:
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [15]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
935it [00:01, 763.65it/s] 
100it [00:00, 1046.16it/s]


In [16]:
batch_size = 256
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [17]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [18]:
model = Sequential()
model.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, to_categorical(y_train), epochs=num_epochs, validation_data=(val_vecs, to_categorical(y_validation)))

Train on 935 samples, validate on 100 samples
Epoch 1/10
935/935 [==============================] - ETA: 1:30 - loss: 1.3174 - acc: 0.312 - ETA: 45s - loss: 1.2192 - acc: 0.359 - ETA: 30s - loss: 1.1794 - acc: 0.37 - ETA: 22s - loss: 1.1845 - acc: 0.35 - ETA: 18s - loss: 1.1753 - acc: 0.36 - ETA: 14s - loss: 1.1869 - acc: 0.34 - ETA: 12s - loss: 1.1743 - acc: 0.34 - ETA: 10s - loss: 1.1746 - acc: 0.36 - ETA: 9s - loss: 1.1560 - acc: 0.3819 - ETA: 8s - loss: 1.1507 - acc: 0.400 - ETA: 7s - loss: 1.1360 - acc: 0.400 - ETA: 6s - loss: 1.1269 - acc: 0.408 - ETA: 5s - loss: 1.1191 - acc: 0.413 - ETA: 5s - loss: 1.1210 - acc: 0.406 - ETA: 4s - loss: 1.1256 - acc: 0.400 - ETA: 4s - loss: 1.1210 - acc: 0.406 - ETA: 3s - loss: 1.1174 - acc: 0.406 - ETA: 3s - loss: 1.1064 - acc: 0.416 - ETA: 2s - loss: 1.1051 - acc: 0.414 - ETA: 2s - loss: 1.1000 - acc: 0.417 - ETA: 2s - loss: 1.0966 - acc: 0.419 - ETA: 1s - loss: 1.0946 - acc: 0.421 - ETA: 1s - loss: 1.0895 - acc: 0.425 - ETA: 1s - loss: 1.0856

935/935 [==============================] - ETA: 2s - loss: 0.3902 - acc: 0.812 - ETA: 2s - loss: 0.3578 - acc: 0.875 - ETA: 2s - loss: 0.3249 - acc: 0.895 - ETA: 2s - loss: 0.3223 - acc: 0.898 - ETA: 2s - loss: 0.3331 - acc: 0.893 - ETA: 2s - loss: 0.3200 - acc: 0.895 - ETA: 2s - loss: 0.3368 - acc: 0.892 - ETA: 2s - loss: 0.3461 - acc: 0.894 - ETA: 2s - loss: 0.3474 - acc: 0.892 - ETA: 2s - loss: 0.3504 - acc: 0.890 - ETA: 2s - loss: 0.3437 - acc: 0.894 - ETA: 1s - loss: 0.3474 - acc: 0.893 - ETA: 1s - loss: 0.3377 - acc: 0.899 - ETA: 1s - loss: 0.3341 - acc: 0.901 - ETA: 1s - loss: 0.3403 - acc: 0.893 - ETA: 1s - loss: 0.3358 - acc: 0.896 - ETA: 1s - loss: 0.3359 - acc: 0.895 - ETA: 1s - loss: 0.3393 - acc: 0.892 - ETA: 1s - loss: 0.3463 - acc: 0.891 - ETA: 1s - loss: 0.3437 - acc: 0.890 - ETA: 0s - loss: 0.3442 - acc: 0.886 - ETA: 0s - loss: 0.3413 - acc: 0.886 - ETA: 0s - loss: 0.3448 - acc: 0.884 - ETA: 0s - loss: 0.3435 - acc: 0.882 - ETA: 0s - loss: 0.3391 - acc: 0.887 - ETA: 0s

In [ ]:
# model.save('./model/bi_lstm_3_pv/bi_lstm_model_01.h5')  

In [19]:
# model = load_model('./model/bi_lstm_3_pv/bi_lstm_model_01.h5')
y_pred = model.predict(val_vecs)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_validation, y_pred, labels = [0, 1, 2], digits=8))

             precision    recall  f1-score   support

          0  0.56060606 0.92500000 0.69811321        40
          1  0.53846154 0.35000000 0.42424242        20
          2  0.76190476 0.40000000 0.52459016        40

avg / total  0.63669664 0.60000000 0.57392983       100

